# LIGN
Graph Induced Lifelong Learning for Spatial-Temporal Data

----

## Imports

In [8]:
import lign as lg
import lign.model as md
import lign.utils.io as io

import torch as th
import torchvision as tv
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

----

## Preprocessing 
The data needs to formatted and set up for lign. A GraphDatabase class is needed to load graphs into the model and track the node. **_Only needs to be ran once_**

### Create Dataset
*_ Only run one of the following cells_ 

*_ Either run "Create Dataset" or "Load Dataset"_

In [3]:
trans = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
dataset = io.cifar_to_lign("data/datasets/CIFAR100", transforms = trans)
dataset.save("data/cifar100.lign")

### Load Dataset
*_ Only run one of the following cells_

*_ Either run "Create Dataset" or "Load Dataset"_

In [9]:
dataset = lg.graph.GraphDataset("data/cifar100.lign")

### Cuda GPUs
Searches for nvidia GPUs in order to speed up training and inference

In [10]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")

### Functions and NNs
Functions and possible NNs used throughout the code to apply changes to the graph's data

In [11]:
def sum_neighs_data(neighs):
    out = neighs[0]
    for neigh in neighs[1:]:
        out = out + neighs
    return out

### Hyperparameters
* Lambda: regulates how much the model relies on difference between the nodes vs the features that lead to their label when calculating loss
* DIST_VEC_SIZE: size of vector representing the mapping of the nodes by the model
* INIT_NUM_LAB: number of labels used to training the model inially in a supervised methods to learn feature mapping
* LABELS: track all the labels that are inputed into the models. The order will be randomized
* SUBGRAPH_SIZE: represent how many nodes to process at once. lign doesn't have batches. This is the closest thing to it

_ LABELS needs to be adjusted to reflect the total number of labels in the model. LABELS is used to track which labels have been given to the the model _

In [12]:
Lambda = 0.0001
DIST_VEC_SIZE = 3
INIT_NUM_LAB = 10
LABELS = np.arange(100)
SUBGRPAH_SIZE = 500


np.random.shuffle(LABELS)

---

## Model
### LIGN
[L]ifelong Learning [I]nduced by [G]raph [N]eural Networks Model (LIGN)

In [13]:
class LIGN_cnn(nn.Module):
    def __init__(self, out_feats):
        super(LIGN_cnn, self).__init__()
        self.gcn1 = md.layers.GCN(func = sum_neighs_data, module_post = nn.Conv2d(3, 6, 5))
        self.gcn2 = md.layers.GCN(func = sum_neighs_data, module_post = nn.Conv2d(6, 16, 5))
        self.gcn3 = md.layers.GCN(func = sum_neighs_data, module_post = nn.Linear(16 * 5 * 5, 150))
        self.gcn4 = md.layers.GCN(func = sum_neighs_data, module_post = nn.Linear(150, 84))
        self.gcn5 = md.layers.GCN(func = sum_neighs_data, module_post = nn.Linear(84, out_feats))
        self.pool = md.layers.GCN(func = sum_neighs_data, module_post = nn.MaxPool2d(2, 2))

    def forward(self, g, features):
        x = self.pool(F.relu(self.gcn1(g, features)))
        x = self.pool(F.relu(self.gcn2(g, x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.gcn3(g, x))
        x = F.relu(self.gcn4(g, x))
        
        return th.tanh(self.gcn5(g, x))

### R-LIGN
[R]ecurrent [L]ifelong Learning [I]nduced by [G]raph [N]eural Networks Model (R-LIGN)

In [ ]:
#dataset.set_data("h", )
#dataset.set_data("c", )

----
## Training